In [ ]:
pip install pycocotools

In [ ]:
import numpy as np                                    # Importa Numpy
import skimage.io as io                               # Importa il modulo Input/ouput di SK-Image
from skimage.transform import resize                  # Importa il modulo resize da SK-Image
from os import listdir                                # Importa il modulo listdir da OS

import json                                           # Importa Json
from matplotlib.collections import PatchCollection    # Importa PatchCollection dal modulo collections di MatPlotLib
from pycocotools.coco import COCO                     # Importa COCO dal modulo coco di PyCoco-Tools
import pycocotools.mask as cocomask                   # Importa il modulo Mask di PyCoco-Tools
import matplotlib.pyplot as plt                       # Importa il modulo  pyplot di MatPlotLib

import PIL
from PIL import Image, ImageDraw                      # Importa il modulo Image da PIL

from tensorflow import keras                          # Importa il modulo Keras di TensorFlow
import os                                             # Importa os

from tqdm import tqdm                                 # Importa il modulo tqdm da tqdm

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.transforms import transforms
from torchvision.models.detection.transform import GeneralizedRCNNTransform

import torch.nn.functional as F

import cv2

import scipy.optimize
import copy
import shutil
import imageio

In [ ]:
# Path
path_in = '/kaggle/input/dataset-ipcv-teama/'
path_out = '/kaggle/working/'

coco_path = path_in + 'COCO_annotations.json/COCO_annotations.json'
train_path = path_in + 'dataset_IPCV/kaggle/working/train'
test_path = path_in + 'dataset_IPCV/kaggle/working/test'
val_path = path_in + 'dataset_IPCV/kaggle/working/val'

weight_path = path_out + 'weight/'

# Path delle predizioni locali
local_crop_img_path = path_out + 'local_crop_image/' # path dei crop delle immagini di test
crop_pred_path = path_out + 'predict/' # path delle predizioni sui crop
output_file_path = path_out + 'local_test_predictions/' # path di salvataggio delle predizioni sui crop
label_path = path_out + 'labels/'

# Path delle predizioni senza labels
nolabels_test_path = '/kaggle/input/xview-dataset-team1/xview-dataset-team1/TestImages'
crop_img_path = path_out + 'crop_image/'
nolabels_output_file_path = path_out + 'test_predictions/'

# Def. funzioni

In [ ]:
def load_image(image_path):
    """
    Carica un'immagine da un percorso specificato.

    Precondizione:
        - `image_path` è il percorso completo dell'immagine da caricare.

    Postcondizione:
        - Restituisce l'immagine caricata.
        - Solleva un'eccezione AssertionError se l'immagine non viene trovata nel percorso specificato.
    """
    image = plt.imread(image_path)
    assert image is not None, f"IMAGE NOT FOUND AT {image_path}"
    return image

In [ ]:
class BuildingsDataset(Dataset):
    def __init__(self, images_dir, annotation_path=None, transform=None):
        """
        Inizializza un oggetto BuildingsDataset.

        Precondizioni:
            - `images_dir` è il percorso della directory contenente le immagini.
            - `annotation_path` è il percorso del file JSON contenente le annotazioni.
            - `transform` è un oggetto trasformazione (es. da torchvision.transforms) per applicare
              trasformazioni alle immagini e alle etichette, opzionale.

        Postcondizioni:
            - `image_paths` è una lista di percorsi completi delle immagini nella directory `images_dir`.
            - `annotations` contiene le annotazioni caricate dal file JSON specificato.
            - `transform` è l'oggetto trasformazione fornito.
        """
        self.image_paths = [os.path.join(images_dir, image_id) for image_id in sorted(os.listdir(images_dir))]
        self.transform = transform
        self.annotation_path = annotation_path
 
        if self.annotation_path is not None:
            with open(annotation_path, 'r') as f:
                self.annotations = json.load(f)
 
    def transform_image_bbox(self, image, label):
        """
        Applica trasformazioni all'immagine e alle etichette, se definite.

        Precondizioni:
            - `image` è l'immagine da trasformare.
            - `label` sono le etichette associate all'immagine.

        Postcondizioni:
            - Se `transform` è definito, applica le trasformazioni all'immagine e alle etichette.
            - Restituisce l'immagine e le etichette trasformate.
        """
        if self.transform:
            transformed = self.transform(image=image, labels=label)
            image = transformed['image']
            label = transformed['labels']
 
        image = transforms.ToTensor()(image)
        return image, label
 
    def __getitem__(self, index):
        """
        Restituisce un campione specifico dell'insieme di dati.

        Precondizioni:
            - `index` è l'indice del campione da restituire.

        Postcondizioni:
            - Restituisce un'immagine e le relative etichette nel formato richiesto.
              Se non ci sono annotazioni per l'immagine, restituisce un target vuoto.
        """
        image_path = self.image_paths[index]
        image_name = image_path.split("/")[-1]
        image = load_image(image_path).astype(np.float32)
        image /= 255.0
        image = torch.from_numpy(image).permute(2,0,1) # change the shape from [h,w,c] to [c,h,w]  

        if self.annotation_path is not None:
            image_id = next((img['id'] for img in self.annotations['images'] if img['file_name'] == image_name), None)
            box_lab = [anno for anno in self.annotations['annotations'] if anno['image_id'] == image_id]

            if(len(box_lab) == 0):
                target = {}
                boxes = torch.zeros((0, 4), dtype=torch.float32) 
                target = {
                        "boxes": boxes, 
                        "labels": torch.zeros(0, dtype=torch.int64), 
                        "image_id": torch.as_tensor([4])
                         }
            else:
                boxes = [json.loads(record['bbox']) for record in box_lab]
                categories = [record['category_id'] for record in box_lab]
                boxes = np.array(boxes)
                # change the co-ordinates into expected [x, y, x+w, y+h] format
                boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
                boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
                boxes = torch.as_tensor(boxes, dtype=torch.float32)

                labels = torch.as_tensor(categories, dtype=torch.int64)

                target = {
                       "boxes": boxes, 
                       "labels": labels,
                       "image_id": torch.tensor([index])
                        }

            return image, target
        else:
            return image, None
        
 
    def __len__(self):
        """
        Restituisce il numero totale di campioni nell'insieme di dati.

        Postcondizioni:
            - Restituisce la lunghezza dell'insieme di dati.
        """
        return len(self.image_paths)

In [ ]:
def get_model(model_name='resnet50', num_classes=60):
    """
    Restituisce un modello Faster R-CNN con una determinata architettura di backbone.

    Precondizioni:
        - `model_name` è una stringa che specifica l'architettura del backbone. 
          Default è 'resnet50'.
        - `num_classes` è il numero di classi dell'insieme di dati. Default è 60.

    Postcondizioni:
        - Restituisce un modello Faster R-CNN con il backbone specificato e il numero di classi.
    """
    backbone = resnet_fpn_backbone(model_name, pretrained=True)
    model = FasterRCNN(backbone, num_classes)
    return model

In [ ]:
def collate_fn(batch):
    """
    Funzione di aggregazione per un batch di campioni.

    Precondizioni:
        - `batch` è una lista di campioni, ciascuno nel formato (immagine, target).
          Dove 'immagine' è un tensore rappresentante l'immagine e 'target' è un dizionario
          contenente le etichette associate all'immagine.

    Postcondizioni:
        - Restituisce una tupla di due elementi:
          1. Un tensore contenente tutte le immagini del batch.
          2. Una lista di dizionari rappresentanti i target corrispondenti alle immagini.
    """
    return tuple(zip(*batch))

In [ ]:
def calculate_ap(precision, recall):
    # Calcola l'area sotto la curva Precision-Recall (AP)
    ap = 0
    for i in range(1, len(precision)):
        ap += (recall[i] - recall[i-1]) * precision[i]
    return ap


def calculate_map(ap_list):
    # Calcola la media delle AP
    mAP = sum(ap_list) / len(ap_list)
    return mAP


def calculate_precision(tp, fp):
    # Calcola la precisione
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    return precision


def calculate_recall(tp, fn):
    # Calcola il recall
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    return recall


def calculate_f1_score(precision, recall):
    # Calcola l'F1 Score
    f1_score = 2 * ((precision * recall) / (precision + recall)) if (precision + recall) != 0 else 0
    return f1_score

In [ ]:
def bbox_iou(boxA, boxB):
    
  # Determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interW = xB - xA
    interH = yB - yA

    # reject non-overlapping boxes
    if interW <=0 or interH <=0 :
        return -1.0

    interArea = interW * interH
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [ ]:
def match_bboxes(bbox_gt, bbox_pred, IOU_THRESH=0.5):
    # Given sets of true and predicted bounding-boxes, determine the best possible match.
    n_true = bbox_gt.shape[0]
    n_pred = bbox_pred.shape[0]
    MAX_DIST = 1.0
    MIN_IOU = 0.0

    # NUM_GT x NUM_PRED
    iou_matrix = np.zeros((n_true, n_pred))
    for i in range(n_true):
        for j in range(n_pred):
            iou_matrix[i, j] = bbox_iou(bbox_gt[i,1:], bbox_pred[j,1:])

    # call the Hungarian matching
    idxs_true, idxs_pred = scipy.optimize.linear_sum_assignment(1 - iou_matrix)

    if (not idxs_true.size) or (not idxs_pred.size):
        ious = np.array([])
    else:
        ious = iou_matrix[idxs_true, idxs_pred]

    ious_actual = iou_matrix[idxs_true, idxs_pred]
    sel_valid = (ious_actual > IOU_THRESH)
    label = sel_valid.astype(int)

    return idxs_true, idxs_pred, ious_actual, label 

In [ ]:
def calculate_metrics(test_img_list, prediction_path, label_path, IOU_THRESH=0.5): 
    tot_true_positive = 0
    tot_false_positive = 0
    tot_false_negative = 0
    
    precision = 0
    recall = 0
    f1 = 0    
    
    # accede alle singole immagini sulle quali è stat fatta predizione
    for elem in test_img_list:
        
        true_positive = 0
        false_positive = 0
        false_negative = 0
        
        elem = elem.split('.')[0]
        elem = elem + '.txt'
        pred_img_path = os.path.join(prediction_path, elem)
        label_img_path = os.path.join(label_path, elem)
        
        if not os.path.exists(pred_img_path):     # se non esite il file delle predizioni, tutti i box della groud truth saranno falsi negativi
            with open(label_img_path, 'r') as file:
                false_negative = len(file.readlines())
                #print(f'TP:{true_positive}, FP:{false_positive}, FN:{false_negative}')
                
        else:   # se essite il file delle predizioni allora bisogna confrontare i box con quelli della ground truth
            with open(pred_img_path, 'r') as boxfile:
                boxes_pred = [line.strip() for line in boxfile]
                boxes_pred = [list(map(float, stringa.split(' '))) for stringa in boxes_pred]
                boxes_pred = np.vstack(boxes_pred)
                
            with open(label_img_path, 'r') as boxfile:
                boxes_label = [line.strip() for line in boxfile]
                boxes_label = [list(map(float, stringa.split(','))) for stringa in boxes_label]
                if len(boxes_label) > 0:
                    boxes_label = np.vstack(boxes_label)
            
            change_coordinate_format(boxes_pred)
            change_coordinate_format(boxes_label)

            match_result = match_bboxes(boxes_label, boxes_pred, IOU_THRESH)   #treshold è il valore di iou oltre il quale c'è un match tra i box (di default è 0.7)
            #print(f'{match_result}\n')
            
            # calcolo falsi negativi e falsi positivi
            if boxes_label.shape[0] > boxes_pred.shape[0]:     # ho più box reali che predetti
                false_negative = boxes_label.shape[0] - boxes_pred.shape[0]
            elif boxes_label.shape[0] < boxes_pred.shape[0]:   # ho più box predetti che reali
                false_positive = boxes_pred.shape[0] - boxes_label.shape[0]
            
            for i in range(len(match_result[0])):
                #print(f'gt:{match_result[0][i]}, pred:{match_result[1][i]}, iou:{match_result[2][i]}, label:{match_result[3][i]}')
            
                index_gt = match_result[0][i]
                index_pred = match_result[1][i]
                if (boxes_label[index_gt,0] == boxes_pred[index_pred,0]) and (match_result[3][i] == 1):
                    true_positive += 1
                else:
                    false_positive += 1
                    false_negative += 1

            #print(f'TP:{true_positive}, FP:{false_positive}, FN:{false_negative}')
       
        # calcolo metriche
        precision += calculate_precision(true_positive, false_positive)
        recall += calculate_recall(true_positive, false_negative)
        f1 += calculate_f1_score(calculate_precision(true_positive, false_positive), calculate_recall(true_positive, false_negative))
    
        tot_true_positive += true_positive
        tot_false_positive += false_positive
        tot_false_negative += false_negative
        
    precision = precision/len(test_img_list)
    recall = recall/len(test_img_list)
    f1 = calculate_f1_score(precision, recall)
    f1 = f1/len(test_img_list)
    
    print(f'precision:{precision}, recall:{recall}, f1_score:{f1}')
            

In [ ]:
def change_coordinate_format(coordinate_list):
    for i,coordinate in enumerate(coordinate_list):
        x_center = coordinate[1]
        y_center = coordinate[2]
        width = coordinate[3]
        heigth = coordinate[4]
        coordinate[1] = x_center - width/2    # x_min
        coordinate[2] = y_center - heigth/2   # y_min
        coordinate[3] = x_center + width/2    # x_max
        coordinate[4] = y_center + heigth/2   # y_max

In [ ]:
def save_labels(test_dataset, label_path, test_img_list):
    labels_list = [test_dataset_elem[1] for test_dataset_elem in test_dataset]

    for i, entry in enumerate(labels_list):
        result_list = []
        labels = entry['labels'].cpu().numpy()
        boxes = entry['boxes'].cpu().numpy()
        for k in range(boxes.shape[0]):
            xmin, ymin, xmax, ymax = boxes[k]
            x_centro = (xmin + xmax) / 2
            y_centro = (ymin + ymax) / 2
            width = xmax - xmin
            height = ymax - ymin
            box_data = np.array([labels[k], x_centro, y_centro, width, height])
            result_list.append(box_data)
            
        np.savetxt(label_path + test_img_list[i].split('.')[0] + '.txt', result_list, fmt='%.6f', delimiter=',')

In [ ]:
def remove_labels(output, soglia_score, soglia_intersezione):   
 
    output_tagliato = []

    for detection in output:
        boxes = detection['boxes']
        labels = detection['labels']
        scores = detection['scores']

        # Trova gli indici dei box che superano la soglia di score
        indici_superati_soglia = scores >= soglia_score

        # Filtra i box, le label e gli score in base agli indici superati la soglia
        boxes_tagliati = boxes[indici_superati_soglia]
        labels_tagliati = labels[indici_superati_soglia]
        scores_tagliati = scores[indici_superati_soglia]

        # Ordina i box in base allo score in ordine decrescente
        indici_ordine_decrescente = torch.argsort(scores_tagliati, descending=True)
        boxes_tagliati = boxes_tagliati[indici_ordine_decrescente]
        labels_tagliati = labels_tagliati[indici_ordine_decrescente]
        scores_tagliati = scores_tagliati[indici_ordine_decrescente]

        # Calcola l'area di intersezione tra tutti i box rimasti
        area_intersezione = torch.zeros(len(scores_tagliati))
        for i in range(len(scores_tagliati)):
            for j in range(i + 1, len(scores_tagliati)):
                # Calcola l'area di intersezione
                x_min = max(boxes_tagliati[i, 0], boxes_tagliati[j, 0])
                y_min = max(boxes_tagliati[i, 1], boxes_tagliati[j, 1])
                x_max = min(boxes_tagliati[i, 2], boxes_tagliati[j, 2])
                y_max = min(boxes_tagliati[i, 3], boxes_tagliati[j, 3])
                area_intersezione[i] = max(0, x_max - x_min) * max(0, y_max - y_min)

        # Trova gli indici dei box con area di intersezione vicina a 1 e tiene solo il box con lo score maggiore
        indici_da_tenere = torch.ones(len(scores_tagliati), dtype=torch.bool)
        for i in range(len(scores_tagliati)):
            for j in range(i + 1, len(scores_tagliati)):
                if area_intersezione[i] / min(area_intersezione[i], area_intersezione[j]) > soglia_intersezione:
                    # Elimina il box con area di intersezione vicina a 1 se ha uno score inferiore
                    if scores_tagliati[i] <= scores_tagliati[j]:
                        indici_da_tenere[i] = False
                    else:
                        indici_da_tenere[j] = False

        # Filtra i box, le label e gli score in base agli indici da tenere
        boxes_tagliati = boxes_tagliati[indici_da_tenere]
        labels_tagliati = labels_tagliati[indici_da_tenere]
        scores_tagliati = scores_tagliati[indici_da_tenere]

        # Creare un nuovo dizionario con i risultati tagliati
        detection_tagliata = {
            'boxes': boxes_tagliati,
            'labels': labels_tagliati,
            'scores': scores_tagliati
        }

        # Aggiungere il risultato alla lista finale solo se ci sono box sopra la soglia
        if len(boxes_tagliati) > 0:
            output_tagliato.append(detection_tagliata)
            
    return output_tagliato

In [ ]:
def calculate_crop(original_width, original_heigth, scarto_sovrapposizione=200):
    #Definizione lista di coordinate per i crop delle immagini (x)
    dim_x = original_width
    x_init = 0
    x_end = 640
    lista_x = []

    while (x_end <= dim_x):
        lista_x.append([x_init, x_end])
        x_init = x_end - scarto_sovrapposizione
        x_end = x_init + 640

    if (x_end > dim_x):
        lista_x.append([x_init, dim_x])

    #Definizione lista di coordinate per i crop delle immagini (y)
    dim_y = original_heigth
    y_init = 0
    y_end = 640
    scarto_sovrapposizione = 200
    lista_y = []

    while (y_end <= dim_y):
        lista_y.append([y_init, y_end])
        y_init = y_end - scarto_sovrapposizione
        y_end = y_init + 640

    if (y_end > dim_y):
        lista_y.append([y_init, dim_y])
    
    return lista_x, lista_y

In [ ]:
def create_crop(img, lista_x, lista_y, crop_img_path):
    lista_img = []

    for elem_y in lista_y:
        for elem_x in lista_x:
            dim_x = elem_x[1] - elem_x[0]
            dim_y = elem_y[1] - elem_y[0]
            if (dim_x == 640) and (dim_y == 640) :
                lista_img.append(img[elem_y[0]:elem_y[1], elem_x[0]:elem_x[1], :])
            else:
                temp = img[elem_y[0]:elem_y[1], elem_x[0]:elem_x[1], :]
                if (dim_x != 640):
                    padding = np.zeros((dim_y, 640-dim_x, 3), dtype=np.uint8)
                    temp = np.concatenate((temp,padding), axis=1)     #concatena l'immagine temp e gli zeri lungo la direzione orizzontale
                    dim_x = 640
                if (dim_y != 640):
                    padding = np.zeros((640-dim_y, dim_x, 3), dtype=np.uint8)
                    temp = np.concatenate((temp,padding), axis=0)     #concatena l'immagine temp e gli zeri lungo la direzione verticale
                    dim_y = 640
                lista_img.append(temp)
                if(temp.shape != (640,640,3)):
                    print('sceeeeem')

    # Save each image in lista_img
    for i, img_chunk in enumerate(lista_img):
        save_path = crop_img_path + 'image_' + str(i) + '.jpg'
        plt.imsave(save_path, img_chunk)

In [ ]:
def aggregate_label(crop_img_path, crop_label_path, output_file_path, posizioni, original_width, original_height):
    for elem in os.listdir(crop_label_path):
        img_name = elem.split('.txt')[0]
        img_path = crop_img_path + img_name + '.jpg'
        img = plt.imread(img_path)
        heigth, width, band = img.shape
 
        with open(crop_label_path + elem, 'r') as file:
            boxes_label = [line.strip() for line in file]
            boxes_label = [list(map(float, stringa.split(','))) for stringa in boxes_label] 
            index = int(img_name.split('_')[-1])
 
            with open(output_file_path, 'a') as file:
                for line in boxes_label:
                    # Sposta le coordinate senza normalizzarle
                    x_center = line[1] + posizioni[index][0]
                    y_center = line[2] + posizioni[index][1]
                    box_width = line[3]
                    box_height = line[4]
 
                    # Scrivi la riga nel formato desiderato
                    file.write(f"{line[0]} {x_center} {y_center} {box_width} {box_height} {index}\n")

In [ ]:
def remove_overlapped_boxes(label_path, treshold=0.25):
    indici_da_eliminare = []

    with open (label_path, 'r') as file:
        boxes = [line.strip() for line in file]
        boxes = [list(map(float, stringa.split())) for stringa in boxes]
        
        #print(boxes[0])

    # boxes_temp contiene i box con le coordinate convertite nel formato: (label,xmin,ymin,xmax,ymax)
    boxes_temp = copy.deepcopy(boxes)
    change_coordinate_format(boxes_temp)
    

    # alcuni box si sovrappongono perché i crop sono con sovrapposizione, quindi applichiamo una sorta di non maximal suppression
    # confrontiamo ciascun box con tutti gli altri della llista (escluso se stesso)
    # se due box sono della stessa classe e il valore di sovrapposizione supera una certa soglia (calcolata con iou) allora il più piccolo dei due viene eliminato
    for i,box in tqdm(enumerate(boxes_temp)):
        for j,box_2 in enumerate(boxes_temp):
            if j != i:
                #print(f'box {box}')
                #print(f'box_2 {box_2}')

                # confrontiamo i box da eliminare solo se provengono da crop diversi (index aggiunto prima), per eliminare solo i box dovuti aalle sovrapposizioni dei crop (uso un basso valore di treshold)
                if box[5] != box_2[5]:      
                    
                    if box[0] == box_2[0]:
                        iou = bbox_iou(box[1:], box_2[1:])
                        if iou >= treshold:
                            area = (box[3] - box[1]) * (box[4] - box[2])    #area = (xmax-xmin)*(ymax-ymin)
                            area_2 = (box_2[3] - box_2[1]) * (box_2[4] - box_2[2])
                            if area_2 <= area:
                                indici_da_eliminare.append(j)
                                
                else:
                    # cerco di eliminare solo i box fortemente sovrapposti (treshold alto) dovuti ad una detection poco precisa del modello 
                    pass

    # la lista di indici è ordinata in modo decrescente, per evitare che durante l'eliminazione ci siano incongruenze tra gli indici
    # (eliminando in ordine crescente, il valore degli indici va a scalare e quindi non si trovano più)
    indici_da_eliminare = list(set(indici_da_eliminare))
    indici_da_eliminare.sort(reverse=True)
    for elem in indici_da_eliminare:
        del boxes[elem]

    # riscriviamo il file dei box,che conterrà ora solo i box non sovrapposti
    with open(label_path, 'w') as file:
        for line in boxes:
            #print(f"{line[0]} {line[1]} {line[2]} {line[3]} {line[4]}\n")
            file.write(f"{line[0]} {line[1]} {line[2]} {line[3]} {line[4]}\n")

In [ ]:
def make_predictions(crop, crop_pred_path, crop_elem):
    model.eval()
    test_image = crop[0].to(device)
    test_image = test_image.unsqueeze(0)
    
    with torch.no_grad():
        output = model(test_image)  
        
    soglia_score = 0.1
    soglia_intersezione = 0.9  # Puoi regolare questa soglia in base alle tue esigenze
    temp = remove_labels(output, soglia_score, soglia_intersezione)

    # Cambio di formato in classe, x_cen, y_cen, width, height
    result_list = []
    for entry in temp:
        labels = entry['labels'].cpu().numpy()
        boxes = entry['boxes'].cpu().numpy()
        for k in range(boxes.shape[0]):
            xmin, ymin, xmax, ymax = boxes[k]
            x_centro = (xmin + xmax) / 2
            y_centro = (ymin + ymax) / 2
            width = xmax - xmin
            height = ymax - ymin
            box_data = np.array([labels[k], x_centro, y_centro, width, height])
            result_list.append(box_data)

    #print(result_list)

    # Salvataggio su file txt
    np.savetxt(crop_pred_path + crop_elem.split('.')[0] + '.txt', result_list, fmt='%.6f', delimiter=',')

In [ ]:
def show_image_with_boxes(image_path, annotation_file):
    # Carica l'immagine come array NumPy
    image_array = imageio.v2.imread(image_path)
    
    # Converti l'array NumPy in un oggetto Image
    image = Image.fromarray(image_array)
    
    # Crea un oggetto ImageDraw per disegnare i bounding box sull'immagine
    draw = ImageDraw.Draw(image)
    
    # Dizionario che mappa classi a colori
    class_colors = {
        48: "yellow",
        5: "green",
    }
    default_color = 'red'

    # Leggi le annotazioni dal file di testo
    with open(annotation_file, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            
            # Calcola le coordinate del bounding box in formato assoluto
            image_width, image_height = image.size
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            color = class_colors.get(int(class_id), default_color)

            # Disegna il bounding box sull'immagine
            draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
            draw.text((x1, y1), f"Class: {int(class_id)}", fill=color)

    # Mostra l'immagine con i bounding box
    plt.figure()
    plt.imshow(image)
    
    return image

# Def. del Modello

In [ ]:
# Crea il modello Faster R-CNN
model_name = 'resnet50'
num_classes = 60
model = get_model(model_name, num_classes)

#print(model)

In [ ]:
# Configurazione della trasformazione per l'oggetto model
model.transform = GeneralizedRCNNTransform(
    min_size=(640,),             # Dimensione minima dell'immagine durante la trasformazione
    max_size=640,                 # Dimensione massima dell'immagine durante la trasformazione
    image_mean=[0.485, 0.456, 0.406],   # Media dell'immagine per la normalizzazione
    image_std=[0.229, 0.224, 0.225]     # Deviazione standard dell'immagine per la normalizzazione
)

In [ ]:
# Sposta il modello sulla GPU, se disponibile
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [ ]:
# Carichiamo i pesi relativi al modello migliore
pesi = '/kaggle/input/dataset-ipcv-teama/best_model_weights_L2rpn_L2roi_10epoch_16batch_0.005lr.pth'
if not torch.cuda.is_available():
    weights = torch.load(pesi, map_location=torch.device('cpu'))
else:
    weights = torch.load(pesi)
model.load_state_dict(weights)

# TESTING 
## Test Locale: faccio predizione sui crop e li rimetto insieme per ottenere la predizione sull'intera immagine, poi calcolo le metriche

In [ ]:
test_dataset = BuildingsDataset(test_path, coco_path)

In [ ]:
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)

if not os.path.exists(local_crop_img_path):
    os.mkdir(local_crop_img_path)

test_img_list = sorted(os.listdir(test_path))
for i,line in enumerate(test_img_list):
    test_img_list[i] = line.split('/')[-1]    # prendo il nome dell'immagine con l'estensione .jpg

for i, elem in enumerate(test_img_list):
    if not os.path.exists(crop_pred_path):
        os.mkdir(crop_pred_path)    
    
    # apriamo l'immagine per ottenere la lunghezza e la larghezza
    img = plt.imread(test_path + '/' + elem)
    original_heigth, original_width, _ = img.shape
    lista_x, lista_y = calculate_crop(original_width, original_heigth, scarto_sovrapposizione=200)
    
    # creazione dei crop per la singola immagine
    crop_save_path = local_crop_img_path + test_img_list[i].split('.')[0] + '/'
    if not os.path.exists(crop_save_path):
        os.mkdir(crop_save_path)
        create_crop(img, lista_x, lista_y, crop_save_path)
    
    test_crop_dataset = BuildingsDataset(crop_save_path)
    crop_list = sorted(os.listdir(crop_save_path))

    for j, crop in enumerate(test_crop_dataset):
        make_predictions(crop, crop_pred_path, crop_list[j])
    
    # viene creato un vettore di posizioni, che tiene conto della posizione di ciascun crop rispetto all'immagine originale (as es. (0,0), (0,640), (640,640), ...)
    # servono per adattare (ovvero spostare) i box calcolati sui crop all'immagine originale (quella grande)
    posizioni = []
    for y,elem_y in enumerate(lista_y):
        for elem_x in lista_x:
            posizioni.append((elem_x[0],elem_y[0]))
                
    # i box dei diversi crop vengono uniti in un unico file, questi box vengono anche adattati all'immagine originale tramite shift e normalizzazione
    test_label_path = output_file_path + elem.split('.')[0] + '.txt'
    aggregate_label(crop_save_path, crop_pred_path, test_label_path, posizioni, original_width, original_heigth)

    if os.path.exists(test_label_path):
        # vengono rimossi i box sovrapposti
        remove_overlapped_boxes(test_label_path, treshold=0.25)
    
    shutil.rmtree(crop_pred_path)

In [ ]:
if not os.path.exists(label_path):
    os.mkdir(label_path)
save_labels(test_dataset, label_path, test_img_list)
calculate_metrics(test_img_list, output_file_path, label_path, IOU_THRESH=0.5)

In [ ]:
test_img_list_tot = sorted(os.listdir(test_path))
test_img_list = []
test_img_list.append(test_img_list_tot[75])
test_img_list.append(test_img_list_tot[63])
for i, elem in enumerate(test_img_list):
    image_final = show_image_with_boxes(test_path + '/' + elem, '/kaggle/working/local_test_predictions/' + elem.split('.')[0] + '.txt')  
    image_final.save(path_out + 'risultati_' + elem.split('.')[0] + '.jpg')

# TESTING - Predizioni sul test set (set di immagini non labellato)

In [ ]:
if not os.path.exists(nolabels_output_file_path):
    os.mkdir(nolabels_output_file_path)

if not os.path.exists(crop_img_path):
    os.mkdir(crop_img_path)

test_img_list_tot = sorted(os.listdir(nolabels_test_path))
test_img_list = []
test_img_list.append(test_img_list_tot[75])
test_img_list.append(test_img_list_tot[63])
test_img_list.append(test_img_list_tot[101])
test_list = test_img_list.copy()

for i,line in enumerate(test_img_list):
    test_img_list[i] = line.split('/')[-1]    # prendo il nome dell'immagine con l'estensione .jpg

for i, elem in enumerate(test_img_list):
    if not os.path.exists(crop_pred_path):
        os.mkdir(crop_pred_path)    
    
    # apriamo l'immagine per ottenere la lunghezza e la larghezza
    img = plt.imread(nolabels_test_path + '/' + elem)
    original_heigth, original_width, _ = img.shape
    lista_x, lista_y = calculate_crop(original_width, original_heigth, scarto_sovrapposizione=200)
    
    # creazione dei crop per la singola immagine
    crop_save_path = crop_img_path + test_img_list[i].split('.')[0] + '/'
    if not os.path.exists(crop_save_path):
        os.mkdir(crop_save_path)
        create_crop(img, lista_x, lista_y, crop_save_path)
    
    test_crop_dataset = BuildingsDataset(crop_save_path)
    crop_list = sorted(os.listdir(crop_save_path))

    for j, crop in enumerate(test_crop_dataset):
        make_predictions(crop, crop_pred_path, crop_list[j])
    
    # viene creato un vettore di posizioni, che tiene conto della posizione di ciascun crop rispetto all'immagine originale (as es. (0,0), (0,640), (640,640), ...)
    # servono per adattare (ovvero spostare) i box calcolati sui crop all'immagine originale (quella grande)
    posizioni = []
    for y,elem_y in enumerate(lista_y):
        for elem_x in lista_x:
            posizioni.append((elem_x[0],elem_y[0]))
                
    # i box dei diversi crop vengono uniti in un unico file, questi box vengono anche adattati all'immagine originale tramite shift e normalizzazione
    test_label_path = nolabels_output_file_path + elem.split('.')[0] + '.txt'
    aggregate_label(crop_save_path, crop_pred_path, test_label_path, posizioni, original_width, original_heigth)

    if os.path.exists(test_label_path):
        # vengono rimossi i box sovrapposti
        remove_overlapped_boxes(test_label_path, treshold=0.25)
    
    shutil.rmtree(crop_pred_path)

In [ ]:
for i, elem in enumerate(test_img_list):
    image_final = show_image_with_boxes(nolabels_test_path + '/' + elem, '/kaggle/working/test_predictions/' + elem.split('.')[0] + '.txt')  
    image_final.save(path_out + 'risultati_' + elem.split('.')[0] + '.jpg')

# Confronto con predizioni su Img intere

In [ ]:
def visualize_results_with_boxes(image_path, results):
    bounding_boxes = results['boxes']

    # Carica l'immagine usando PIL
    image = Image.open(image_path)

    # Inizializza ImageDraw con l'immagine
    draw = ImageDraw.Draw(image)

    # Disegna i bounding box sull'immagine
    for bbox in bounding_boxes:
        bbox_coords = bbox  # Converti la stringa bbox in una lista di coordinate
        bbox_coords = [bbox_coords[0],bbox_coords[1],bbox_coords[2],bbox_coords[3]]
        draw.rectangle(bbox_coords, outline='red', width=2)

    # Visualizza l'immagine con i bounding box
    plt.imshow(image)
    #plt.show()
    return image

def visualize_image_with_boxes(image_path, coco_annotations_path):
    # Carica il file COCO annotations JSON
    with open(coco_annotations_path, 'r') as coco_file:
        coco_data = json.load(coco_file)

    # Estrai l'id dell'immagine dal nome del file
    image_name = image_path.split("/")[-1]
    image_id = next((img['id'] for img in coco_data['images'] if img['file_name'] == image_name), None)

    # Se l'id dell'immagine è trovato, estrai i bounding box corrispondenti
    if image_id is not None:
        bounding_boxes = [bbox for bbox in coco_data['annotations'] if bbox['image_id'] == image_id]
        
        # Carica l'immagine usando PIL
        image = Image.open(image_path)

        # Inizializza ImageDraw con l'immagine
        draw = ImageDraw.Draw(image)

        # Disegna i bounding box sull'immagine
        for bbox in bounding_boxes:
            bbox_coords = eval(bbox['bbox'])  # Converti la stringa bbox in una lista di coordinate
            bbox_coords = [bbox_coords[0],bbox_coords[1],bbox_coords[0]+bbox_coords[2],bbox_coords[1]+bbox_coords[3]]
            draw.rectangle(bbox_coords, outline='red', width=2)

        # Visualizza l'immagine con i bounding box
        plt.imshow(image)
        #plt.show()
        return image
    else:
        print(f"Image {image_name} not found in COCO annotations.")

## test locale (con labels)

In [ ]:
test_images = sorted(os.listdir(test_path))
indici = [75,63]

for indice in indici:
    # Test su un'immagine
    model.eval()
    test_image = test_dataset[indice][0]

    test_image = test_image.to(device)
    test_image = test_image.unsqueeze(0)
    with torch.no_grad():
        output = model(test_image) 

    soglia_score = 0.1
    soglia_intersezione = 0.85  # Puoi regolare questa soglia in base alle tue esigenze

    output_tagliato = remove_labels(output, soglia_score, soglia_intersezione)
    elem = test_images[indice]

    original_image_path = test_path + '/' + elem
    plt.figure(figsize=(10,10))    
    img = visualize_results_with_boxes(original_image_path, output_tagliato[0])
    img.save(path_out + 'pred_intera_' + elem.split('.')[0] + '.jpg')

## test senza labels

In [ ]:
test_images = sorted(os.listdir(nolabels_test_path))
indici = [75,63,101]

nolabels_test_dataset = BuildingsDataset(nolabels_test_path)
for indice in indici:
    # Test su un'immagine
    model.eval()
    test_image = nolabels_test_dataset[indice][0]

    test_image = test_image.to(device)
    test_image = test_image.unsqueeze(0)
    with torch.no_grad():
        output = model(test_image) 

    soglia_score = 0.1
    soglia_intersezione = 0.85  # Puoi regolare questa soglia in base alle tue esigenze

    output_tagliato = remove_labels(output, soglia_score, soglia_intersezione)
    elem = test_images[indice]

    original_image_path = nolabels_test_path + '/' + elem
    plt.figure(figsize=(10,10))    
    img = visualize_results_with_boxes(original_image_path, output_tagliato[0])
    img.save(path_out + 'pred_intera_' + elem.split('.')[0] + '.jpg')